# Regression and Prediction

In [64]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as sm

## Load up some data

In [8]:
tips = sns.load_dataset("tips")
tips.rename(columns={'smoker': 'drinker', 'sex': 'gender'}, inplace=True) # Just a little column renaming

In [9]:
# Describe what we want using R-style formulas
formula = 'tip ~ total_bill + size + C(gender) + C(drinker) + C(day) + C(time)'
model = sm.ols(formula, data=tips)    # Describe model
results = model.fit()                 # Fit model       
results.summary()                     # Summarize model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     26.06
Date:                Fri, 11 Mar 2016   Prob (F-statistic):           1.20e-28
Time:                        21:32:07   Log-Likelihood:                -347.48
No. Observations:                 244   AIC:                             713.0
Df Residuals:                     235   BIC:                             744.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept               0.5908      0.256      2.310      0.022         0.087     1.095
C(gender)[T.Female]     0.0324      0.142      0.229      0.819        -0.247     0.311
C(drinker)[T.No]        0.0864      0.147      0.589      0.556        -0.202     0.375
C(day)[T.Fri]           0.1623      0.393      0.412      0.680        -0.613     0.937
C(day)[T.Sat]           0.0408      0.471      0.087      0.931        -0.886     0.968
C(day)[T.Sun]           0.1368      0.472      0.290      0.772        -0.793     1.066
C(time)[T.Dinner]      -0.0681      0.445     -0.153      0.878        -0.944     0.808
total_bill              0.0945      0.010      9.841      0.000         0.076     0.113
size                    0.1760      0.090      1.966      0.051        -0.000     0.352
==============================================================================
Omnibus:                       27.860   Durbin-Watson:                   2.096
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.555
Skew:                           0.607   Prob(JB):                     3.87e-12
Kurtosis:                       4.923   Cond. No.                         281.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
columns = ['total_bill', 'gender', 'drinker', 'day', 'time', 'size']
data = [[15.52, 'Female', 'No', 'Sun', 'Lunch', 1]]
df = pd.DataFrame(data, columns=columns)
predictions = results.predict(df).tolist()  # Convert to simple list

In [62]:
predictions

[2.488895559955752]

In [63]:
from requests import get, post
request_data = [
	{"total_bill": 47.26, "gender": "Female", "drinker": "Yes", "day": "Sat", "time": "Dinner", "size": 3},
	{"total_bill": 38.73, "gender": "Female", "drinker": "Yes", "day": "Sat", "time": "Dinner", "size": 3},
	{"total_bill": 42.50, "gender": "Female", "drinker": "Yes", "day": "Sat", "time": "Dinner", "size": 3},
]
post('http://localhost:5000/', json=request_data).json()

{u'bill': 128.49, u'predicted_tip': 15.51, u'tip_percentage': 0.12}